In [1]:
from bs4 import BeautifulSoup
from astropy.table import QTable, Table
import numpy as np
import astropy.units as u
import pandas as pd
from astropy.coordinates import SkyCoord
from pathlib import Path
from glob import glob
import datetime

save = True

In [ ]:
wanted_columns = [
    "SIMBAD",
    "HD",
    "HIP",
    "SpType",
    "RotVel", # km/s, v sin(i)
    "V",
    "H",
    "K",
    "LDD",
    "e_LDD",
    "e_LDD_rel",
    "UD_V",
    "UD_H",
    "UD_K",
    "vis2",
    "vis2Err",
]

optimal_dra = 0.5 * u.hourangle

max_baseline = 330 * u.m
obs_wavelength = 1.65 * u.um
min_scale = ((obs_wavelength / max_baseline) * u.rad).to(u.mas)
#print(min_scale)

searchcal_out_table_dir = Path("./searchcal_output/")
sifted_table_dir = Path("./sifted_tables/")
sifted_table_pattern = sifted_table_dir / "stem.ecsv"

searchcal_tables = glob(str(searchcal_out_table_dir/"*.csv"))
print(searchcal_tables)

wiki_wanted_columns = [ # Column name, unit, precision
    ["SIMBAD", None, None],
    ["HD", None, 0],
    ["HIP", None, 0],
    ["Sep", u.deg, 2],
    ["RASep", u.hourangle, 2],
    ["DECSep", u.deg, 2],
    ["OPTSep", u.deg, 2],
    ["SpType", None, None],
    ["RotVel", None, 1], # km/s, v sin(i)
    ["V", None, 2],
    ["H", None, 2],
    ["K", None, 2],
    ["LDD", None, 3],
    ["e_LDD", None, 3],
    ["e_LDD_rel", None, 3],
    ["UD_V", None, 3],
    ["UD_H", None, 3],
    ["UD_K", None, 3],
    ["vis2", None, 3],
    ["vis2Err", None, 3],
    ["Scores", None, 1],    
]

['searchcal_tables/searchcal_output/del_leo.csv', 'searchcal_tables/searchcal_output/bet_tau.csv', 'searchcal_tables/searchcal_output/eta_uma.csv', 'searchcal_tables/searchcal_output/alf02_cvn.csv', 'searchcal_tables/searchcal_output/bet_cmi.csv', 'searchcal_tables/searchcal_output/bet_cma.csv', 'searchcal_tables/searchcal_output/bet_leo.csv', 'searchcal_tables/searchcal_output/alf_cep.csv', 'searchcal_tables/searchcal_output/bet_eri.csv', 'searchcal_tables/searchcal_output/alf_lep.csv', 'searchcal_tables/searchcal_output/bet_lib.csv', 'searchcal_tables/searchcal_output/rho_pup.csv', 'searchcal_tables/searchcal_output/eta_tau.csv', 'searchcal_tables/searchcal_output/bet_uma.csv', 'searchcal_tables/searchcal_output/gam_ori.csv', 'searchcal_tables/searchcal_output/del_crv.csv', 'searchcal_tables/searchcal_output/zet_aql.csv', 'searchcal_tables/searchcal_output/zet_oph.csv', 'searchcal_tables/searchcal_output/eps_uma.csv', 'searchcal_tables/searchcal_output/kap_ori.csv', 'searchcal_tables

In [ ]:
for t in searchcal_tables:
    df = pd.read_csv(t, comment = "#")
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

    searchcal_table = Table(df.values, names = df.columns)
    coords = []
    dists = []
    ra_dists = []
    dec_dists = []
    optimal_point_dists = []
    row1 = searchcal_table[0]
    row1_coord = SkyCoord(row1["RAJ2000"], row1["DEJ2000"], unit = (u.hourangle, u.deg))
    optimal_points = [
        SkyCoord(ra=row1_coord.ra - optimal_dra, dec=row1_coord.dec),
        SkyCoord(ra=row1_coord.ra + optimal_dra, dec=row1_coord.dec),        
    ]

    for row in searchcal_table:
        coord = SkyCoord(row["RAJ2000"], row["DEJ2000"], unit = (u.hourangle, u.deg))
        coords.append(coord)
        dist = row1_coord.separation(coord)
        dists.append(dist)
        ra_dist = (coord.ra - row1_coord.ra).to(u.hourangle)
        ra_dists.append(ra_dist)
        dec_dist = (coord.dec - row1_coord.dec).to(u.deg)
        dec_dists.append(dec_dist)

        if np.isclose(dist, 0):
            optimal_point_dists.append(np.nan * u.deg)
            continue
        
        optimal_seps = []
        for p in optimal_points:
            optimal_seps.append(p.separation(coord).to(u.deg).value)
        optimal_point_dists.append(np.amin(optimal_seps) * u.deg)

    eval_table = searchcal_table[wanted_columns]

    eval_table.add_columns((coords, dists, ra_dists, dec_dists, optimal_point_dists), indexes = (3, 3, 3, 3, 3), names = ("RADEC", "Sep", "RASep", "DECSep", "OPTSep"))

    cut_table = eval_table[eval_table['Sep'] <= 10 * u.deg]

    new_HDs = []
    for hd in cut_table['HD']:
        new_HDs.append("{:0.0f}".format(hd).strip("."))
    cut_table['HD'] = new_HDs

    def CalibratorScore(row):
        # Separation from optimal calibrator placement
        ra_score = np.abs(optimal_dra.value - np.abs(row['RASep']))
        ra_scale = 1.0
        
        optsep_score = 10 - row['OPTSep']
        optsep_scale = 1.0

        # Rotational Velocity Score
        rotvel_score = float(row['RotVel'] < 200)
        rotvel_scale = 1.0

        # UDD Score
        udd_score = float(row['UD_H'] < min_scale.to(u.mas).value)
        udd_scale = 1.0

        # LDD Score
        ldd_score = float(row['LDD'] < min_scale.to(u.mas).value)
        ldd_scale = 1.0

        # Generate final score
        score = 0
        score += ra_score * ra_scale
        score += optsep_score * optsep_scale
        score += rotvel_score * rotvel_scale
        score += udd_score * udd_scale
        score += ldd_score * ldd_scale

        return score

    scores = []
    score_breakdowns = []
    for idx, row in enumerate(cut_table):
        if idx == 0:
            scores.append(np.inf)
            continue
        
        score = CalibratorScore(row)
        scores.append(score)

    cut_table.add_column(scores, index = -1, name = "Scores")

    cut_table.sort("Scores", reverse = True)

    outfname = Path(t).stem

    outpath = sifted_table_pattern.with_stem(outfname)

    if save:
        cut_table.write(str(outpath), overwrite = True)

    mkdown_out = cut_table.to_pandas().to_markdown(index = False)

    mkdown_outpath = outpath.with_suffix(".md")

    if save:
        with open(mkdown_outpath, 'w') as f:
            f.write(mkdown_out)

    # The below was generated using GPT-4o and modified

    # Function to convert DataFrame to MediaWiki table format
    def is_float_like(value):
        # Check if the value is a float or a numpy float type
        if isinstance(value, float):
            return True
        elif isinstance(value, np.float32) or isinstance(value, np.float64):
            return True
        elif isinstance(value, np.generic):
            # Check if it's a numpy scalar and not an integer
            return not isinstance(value, (np.int32, np.int64, np.int8, np.int16, np.intc))
        
        return False
    
    def df_to_mediawiki(df):
        # Start with the table header
        mediawiki_table = "== Interferometric Calibrator Table - CHARA ==\n"
        mediawiki_table += "{| class=\"wikitable\"\n"
        
        # Add the header row
        unit_string = "\nUnits -"
        mediawiki_table += "|+\n"
        for c in wiki_wanted_columns:
            mediawiki_table += "!{}\n".format(c[0])
            if c[1] is not None:
                unit_string += " {}: {}".format(c[0], str(c[1]).strip())


        # Add each row of the DataFrame
        for index, row in df.iterrows():
            mediawiki_table += "|-\n"
            wiki_wc_last = len(wiki_wanted_columns) - 1
            for idx, c in enumerate(wiki_wanted_columns):
                name = c[0]
                prec = c[2]

                value = row[name]
                
                if prec is None:
                    valstr = str(value)
                elif prec == 0:
                    valstr = str(round(float(value), prec)).split(".")[0]
                else:
                    valstr = str(round(float(value), prec))
                
                

                if idx != wiki_wc_last:
                    mediawiki_table += "| {} |".format(valstr)
                else:
                    mediawiki_table += "| {}\n".format(valstr)

#            rowlen = len(row)
#            for idx, value in enumerate(row):
#                if idx == rowlen-1:
#                    if is_float_like(value):
#                        mediawiki_table += "| " + str(np.round(value, 3)) + "\n"
#                    else:
#                        mediawiki_table += "| " + str(value) + "\n"
#                else:
#                    if is_float_like(value):
#                        mediawiki_table += "| " + str(np.round(value, 3)) + " |"
#                    else:
#                        mediawiki_table += "| " + str(value) + " |"
        
        # End the table
        mediawiki_table += "|}"

        generator_link = "https://drive.google.com/file/d/16YFkWxVqMv6mvf3FU8qOdNuLHu1IMyWc/view?usp=sharing"
        generator_time = datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%d")
        mediawiki_table += "\n{}".format(unit_string)
        mediawiki_table += "\n\nTable generated on {} using [{} SearchCal settings specified here].".format(generator_time, generator_link)
        
        return mediawiki_table

    # Convert the DataFrame to MediaWiki format
    mediawiki_output = df_to_mediawiki(cut_table.to_pandas())
    # Write the output to a text file
    if not save:print(mediawiki_output)
    if save:
        with open(outpath.with_suffix(".txt"), 'w') as f:
            f.write(mediawiki_output)